# Exercise 4.8
Grouping and Aggregating Data

Content List:
1. Import Libraries, Create Path and Import Datasets
2. Find the aggregated mean of 'order_number" grouped by 'department_id"
3. Create Customer Loyalty Flags
4. Analyze statistics with new Loyalty Flag
5. Create Spending Flags
6. Create Order Frequency Flags
7. Export Updated Dataset

1. Import Libraries, Create Path and Import Datasets

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
#Create Path
path = r'C:\Users\ryanc\OneDrive\Documents\Data Analytics Exercises\Python\Instacart Basket Analysis'

In [4]:
#Import order products merged updated df
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_updated.pkl'))

In [20]:
#View first 5 rows of dataset
ords_prods_merged.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,new_busiest_days,busiest_hours,max_order,loyalty_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,3.0,5,0,both,Mid-range product,Regularly busy,Regularly Busy days,Most Orders,32,Regular customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,20.0,1,1,both,Mid-range product,Regularly busy,Regularly Busy days,Average Orders,32,Regular customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,6.0,20,0,both,Mid-range product,Busiest day,Busiest days,Average Orders,5,New customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,NaN,10,0,both,Mid-range product,Regularly busy,Slowest Days,Most Orders,3,New customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,9.0,11,1,both,Mid-range product,Least busy,Slowest Days,Average Orders,3,New customer


# Question 2. 
Find the aggregated mean of 'order_number" grouped by 'department_id"

In [5]:
#Find the aggregated mean of 'order_number" grouped by 'department_id"
ords_prods_merged.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

# Question 3.
Looking back on the numbers for the subset of this data used previously, these numbers, using the entire data set are very close to the subset of data we used. This is expected that they would be close but not exact as our subset was considerably large at 1,000,000 rows. 

3. Create Customer Loyalty Flags

In [6]:
#Use the transform function to create a new column 'max_order'
ords_prods_merged['max_order'] = ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

# Question 4.
Create a loyalty flag

In [7]:
#Create flag for loyal customers (next 3 lines)
ords_prods_merged.loc[ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
ords_prods_merged.loc[(ords_prods_merged['max_order'] <= 40) & (ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [9]:
ords_prods_merged.loc[ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
#View count of new loyalty flag
ords_prods_merged['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [25]:
#View selected columns sorted by user_id
ords_prods_merged[['user_id', 'order_number', 'max_order', 'loyalty_flag']].sort_values(by=['user_id']).head(10)

,user_id,order_number,max_order,loyalty_flag
6002731,1,5,10,New customer
92374,1,7,10,New customer
92373,1,6,10,New customer
92371,1,4,10,New customer
92370,1,3,10,New customer
92369,1,2,10,New customer
7157235,1,8,10,New customer
92368,1,1,10,New customer
7157236,1,9,10,New customer
7157237,1,10,10,New customer


4. Analyze statistics with new Loyalty Flag

# Question 5
Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category

In [18]:
#View statistics about prices of products purchased grouped by loyalty
ords_prods_merged.groupby('loyalty_flag')['prices'].describe()

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,10284093.0,10.386336,328.017787,1.0,4.2,7.4,11.2,99999.0
New customer,6243990.0,13.294670,597.560299,1.0,4.2,7.4,11.3,99999.0
Regular customer,15876776.0,12.495717,539.720919,1.0,4.2,7.4,11.3,99999.0


Looking at the descriptive statistics for each loyalty category we see that new customers have the highest mean price per item than both regular and loyal customers. Loyal customers spend the least per item on average.

5. Create Spending Flags

# Question 6. 
Create a spending flag for each user based on the average price across all their orders using the following criteria

In [21]:
#Use the transform function to create a new column 'avg_price'
ords_prods_merged['avg_price'] = ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [22]:
#Create flag for high and low spenders (next 2 lines)
ords_prods_merged.loc[ords_prods_merged['avg_price'] < 10, 'spender_flag'] = 'Low Spender'

In [23]:
ords_prods_merged.loc[ords_prods_merged['avg_price'] >= 10, 'spender_flag'] = 'High Spender'

In [24]:
#View frequency of new spender flag
ords_prods_merged['spender_flag'].value_counts(dropna = False)

spender_flag
Low Spender     31770614
High Spender      634245
Name: count, dtype: int64

6. Create Order Frequency Flags

# Question 7.
Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column

In [26]:
#Use the transform function to create a new column 'order_freq'
ords_prods_merged['order_freq'] = ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [27]:
#Create flag for frequency of customer (next 3 lines)
ords_prods_merged.loc[ords_prods_merged['order_freq'] > 20, 'frequency_flag'] = 'Non-Frequent Customer'

In [28]:
ords_prods_merged.loc[(ords_prods_merged['order_freq'] > 10) & (ords_prods_merged['order_freq'] <=20), 'frequency_flag'] = "Regular Customer"

In [30]:
ords_prods_merged.loc[ords_prods_merged['order_freq'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [31]:
#View count of new frequency flag
ords_prods_merged['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent Customer        21559853
Regular Customer          7208564
Non-Frequent Customer     3636437
NaN                             5
Name: count, dtype: int64

7. Export Updated Dataset

In [32]:
#Export Data
ords_prods_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_grouped.pkl'))